
# Gen-Toolbox data analyis notebook

This notebook is part of the gen-toolbox project, a comprehensive tool designed for collating large numbers of VCF files from unique samples, annotating variants, and creating variant frequency tables. The project is tailored to run on substantial servers and has been spearheaded by the Tartu University Hospital Centre of Medical Genetics and the Tartu University Institute of Clinical Medicine, with the backing of the Estonian Research Council grant PSG774.

## Notebook Objective:

This particular notebook delves into the statistical analysis of Single Nucleotide Variants (SNVs) in genomic data. The aim is to:

- Load and process the combined frequency tables created from a large number of (g)VCF samples with the main Hail wrapper.
- Compute statistical metrics and burden ratios.
- Visualize results for a graphical analysis of the data.

## Prerequisites:

1. **Data Preparation**: Ensure you have the necessary VCF files and gene configurations. The notebook expects CSV or TSV formatted files.

2. **Environment Setup**: If using Docker, ensure Docker is installed and running. Alternatively, ensure you have a Python environment set up with all necessary libraries.

3. **Configuration**: Adjust path variables in the notebook to match the location of your data files. Jupyter notebooks detect data only from the same folder as the notebook itself (due to portability of notebooks)

### 0. Library Imports

In [ ]:
import json
import pathlib
import re

import scipy.stats as sp

import matplotlib.pyplot as plt
from adjustText import adjust_text
import numpy as np
# Pandas version 1.24 required for hail, significant speed improvement in 2.0.X however
import pandas as pd
# Use modin as a pandas substitute for improved speeds
#import modin.pandas as pd

### 1. Input variables

#### 1.1 Path variables - change these for specifing correct paths

In [ ]:
gene_config = "/mnt/c/Users/ville/Documents/PycharmProjects/gen-toolbox/src/config/gene_config.json"
args_case =  "/mnt/c/Users/ville/OneDrive - Tartu Ülikool/Doktorantuur/Oligogeensus/Frequency_databases/frequency_table_643_LIHAS_positive.csv" # add here TSV or CSV file with case data
args_control = "/mnt/c/Users/ville/OneDrive - Tartu Ülikool/Doktorantuur/Oligogeensus/Frequency_databases/frequency_table_9099_LIHAS_negative.csv" # add here TSV or CSV file with control data

#### 1.2 Configuration variables

In [ ]:
is_csv = True # if False, then tsv, if True, then csv
iterations = 300000 # number of iterations for permutation test
combination_length=3 # number of genes in a set
case_genes_length = combination_length  # e.g. sets of 5 genes


## 2. Utility functions

In [ ]:
def load_gene_config(json_file):
    """Load gene configuration from a JSON file."""
    p = pathlib.Path(json_file)
    config = json.loads(p.read_bytes())
    return config

In [ ]:
def extract_number_from_filename(filename: str) -> int:
    """Extract the number from the given filename."""
    match = re.search(r"\d+", filename)
    return int(match.group()) if match else None

In [ ]:
case_count = extract_number_from_filename(args_case) # extract number from filename
control_count = extract_number_from_filename(args_control) # extract number from filename


## 3. Data Loading

### 3.1 Loading Gene config

In [ ]:
config = load_gene_config(gene_config)

In [ ]:
# keys in gene_config would show information present in it
config.keys()

In [ ]:
# Lets Show some of itersect genes
config['intersect_genes_tso'][:5]

In [ ]:
# How many intersect genes are there
len(config['intersect_genes_tso'])

In [ ]:
# printing number of genes in each conf
for conf in [config["intersect_genes_tso"], config["intersect_genes_tshc"]]:
    print(len(conf))

### 3.2 Getting rv_genes and neg_control_genes for use in our test

In [ ]:
rv_genes = config["rv_genes"]
neg_control_genes = config["neg_control_genes"]

### 3.3 Loading case and control SNV data

For csv data file intersect_genes_tso is loaded while for tsv intersect_genes_tshc is loaded. Read the docs for clarification

In [ ]:
if is_csv:
    intersect_genes = config["intersect_genes_tso"]
    df_case = pd.read_csv(args_case, sep=",", header=0)
    df_control = pd.read_csv(args_control, sep=",", header=0)
else:
    intersect_genes = config["intersect_genes_tshc"]
    df_case = pd.read_table(args_case, sep="\t", header=0)
    df_control = pd.read_table(args_control, sep="\t", header=0)

As we have loaded single file for case and control, both dataframes will be same

In [ ]:
df_case.head(2)

In [ ]:
df_control.head(2)

In [ ]:
# Dataframe length
# Each row contains a gene
f'number of genes in case: {len(df_case)}', f'number of genes in control: {len(df_control)}'

### 3.4 Assigning the 1st column the name of gene

In [ ]:
df_case.rename(columns={df_case.columns[0]: "gene"}, inplace = True)
df_control.rename(columns={df_control.columns[0]: "gene"}, inplace = True)

In [ ]:
df_case.columns

### 3.5 Getting list of all genes

In [ ]:
all_genes = df_case['gene']

## 4. Cleaning data

### 4.1 Selecting intersect gene from config

In [ ]:
# Selecting only itersecting genes from config
if intersect_genes is not None:
    # Filter out empty genes and keep only the intersecting genes in both dataframes
    df_case = df_case.dropna(subset=["gene"])
    df_case = df_case[df_case.gene.isin(intersect_genes)]
    df_control = df_control.dropna(subset=["gene"])
    df_control = df_control[df_control.gene.isin(intersect_genes)]
    df_case.reset_index(drop=True, inplace=True)
    df_control.reset_index(drop=True, inplace=True)

In [ ]:
# Dataframe length after selecting only intersecting genes available in config
f'number of genes in case: {len(df_case)}', f'number of genes in control: {len(df_control)}'

In [ ]:
#Check if dataframes are of equal length
if len(df_case.index) != len(df_control.index):
    print("WARNING: Case dataframe length does not match control dataframe length! The intersect of both dataframes will be analysed.")

### 4.2 Taking intersect of both data frames and sorting on gene column

This step is performed to ensure that same genes for case and control are present and we can apply indexwise operation in our statistical test. Randomly selected indices will produce same genes from both data frames.

Sorting is done because if we select gene names randomly and then filter both dataframes for selected gene names then it will be a slow process


In [ ]:
# Take only the intersect of two dataframes based on gene column
intersection_values = set(df_case['gene']).intersection(df_control['gene'])
df_case = df_case[df_case["gene"].isin(intersection_values)]
df_case = df_case.sort_values(by="gene")
df_case = df_case.fillna(0)
df_control = df_control[df_control["gene"].isin(intersection_values)]
df_control = df_control.sort_values(by="gene")
df_control = df_control.fillna(0)
df_case.reset_index(drop=True, inplace=True)
df_control.reset_index(drop=True, inplace=True)

In [ ]:
# The rows must match in order to do index based math
assert df_case["gene"].equals(df_control["gene"]), "Case and control dataframe indices do not match!"

## 5. Statistical test

### 5.1 Initializing empty dataframes to store results

In [ ]:
fraction_results_1 = pd.DataFrame()
fraction_results_2 = pd.DataFrame(columns=df_case.columns[1:].tolist())

### 5.2 Calculating desired variables

If we have passed only 1 file it will make expected_ratio to be 1

In [ ]:
expected_ratio = case_count / control_count
df_case.reset_index(drop=True, inplace=True)
df_control.reset_index(drop=True, inplace=True)
columns_to_add = df_case.columns[1:]

# Calculate the mean of the case and control dataframes
df_case_mean = df_case.mean()
df_control_mean = df_control.mean()

num_columns = df_case.shape[1]

print("Case means \n{0}\nControl means \n{1}".format(df_case_mean, df_control_mean))
print("Expected ratio cases / controls: {0}, log2 {1}".format(expected_ratio, np.log2(expected_ratio)))
print("Expected ratio cases / controls by group (log2): \n {0}".format(np.log2(df_case_mean / df_control_mean)))

In [ ]:
# Divisible columns, look at the ratios grossly
divison_result_gross = df_case[columns_to_add] / df_control[columns_to_add]
r = divison_result_gross[divison_result_gross[:-1] > expected_ratio].dropna(how="all")
r["gene"] = df_case["gene"]  ## Do nothing

### 5.2 Random selection of indices for statistical test

In [ ]:
indices = np.array([np.random.choice(df_case.index, size=case_genes_length, replace=False) for _ in range(iterations)])

### 5.3 Taking sum for columns against selected indices for each iteration

In [ ]:
# Calculate the sum of total variants for the case and control groups using the sampled indices
total_variants_case = np.array([df_case.iloc[idx, 1:].sum().to_numpy() for idx in indices])
total_variants_control = np.array([df_control.iloc[idx, 1:].sum().to_numpy() for idx in indices])

In [ ]:
# Show what a single burden event looks like
total_variants_case
total_variants_control

### 5.4 Ratios vector for each column based on given condition to include sum

In [ ]:
ratios_vector = np.where(np.logical_and(total_variants_case > 1, total_variants_control > 1),\
                total_variants_case / total_variants_control,\
                np.NaN)

In [ ]:
# numer of iterations and column number can be seen in shape
ratios_vector.shape

In [ ]:
# Where sum is less than 1 NaN will be placed for the ratio in that
ratios_vector[0]

### 5.5 Satisfying below statistical condition to get burden events
Low variant count events are not statistically significants.
I.e. gene sets containing only a few variants or variants in only one gene are excluded.
High variant burdens are expected between impactful genes in the different groups.



1. $$ \log_2\left(\frac{\text{ratios\_vector}}{\text{expected\_ratio}}\right) > log2(99th\_percentile) $$


2. $$ \text{total\_variants\_case} > \text{case\_genes\_length} $$


3. $$ \text{total\_variants\_control} > \text{case\_genes\_length} $$

4. $$ \text{ Nonzero values total\_variants\_control} > 1 $$

5. $$ \text{ Nonzero values total\_variants\_case} > 1 $$


In [ ]:
# Remove rows where there are only single values in a burden event (i.e. [0, 0, 0, 0.4])
# Because we are looking for interactions between genes
def count_zeros(arr):
    return np.greater_equal(sum(1 for x in arr if (x == 0 or np.isnan(x))), combination_length-1)

In [ ]:
fraction_results_tmp = pd.DataFrame(ratios_vector, columns=df_case.columns[1:].tolist())

complex_condition = (
    (np.log2(ratios_vector/expected_ratio) > 0.2) & # Early filtering to reduce about 75% of data
    (total_variants_case > case_genes_length) &
    (total_variants_control > case_genes_length) &
    (np.count_nonzero(~np.isnan(total_variants_control)) > 1) &
    (np.count_nonzero(~np.isnan(total_variants_case)) > 1)
                    ).T.flatten()

gene = np.array([df_case.gene[idx].to_numpy() for idx in indices])
gene = np.tile(gene, (len(df_case.columns[1:].tolist()),1))
case = np.array([df_case.iloc[idx, 1:].to_numpy().T for idx in indices]).transpose((1, 0, 2))
control = np.array([df_control.iloc[idx, 1:].to_numpy().T for idx in indices]).transpose((1, 0, 2))

case = case.reshape((-1, case.shape[-1]))
control = control.reshape((-1, control.shape[-1]))
case_control = case/control
gene_case_control = np.dstack((gene, case_control)) # Make a nested object combining ratios to genes, exploded later in section 6.0

In [ ]:
fraction_results_1 = pd.DataFrame()
fraction_results_1['frequency_bin'] = np.repeat(fraction_results_tmp.columns, len(fraction_results_tmp))
fraction_results_1['burden_ratio'] = fraction_results_tmp.to_numpy().T.flatten()
fraction_results_1["burden_ratio_norm"] = fraction_results_1['burden_ratio'].div(expected_ratio)
fraction_results_1["burden_ratio_norm_log2"] = np.log2(fraction_results_1['burden_ratio_norm'])
fraction_results_1['gene'] = list(gene)
fraction_results_1['case'] = list(case)
fraction_results_1['control'] = list(control)
fraction_results_1['case_control'] = list(case_control)
fraction_results_1['case_control_gene'] = list(gene_case_control)

# Filter out a large amount of low ratio or low variant count burdens
fraction_results_1 = fraction_results_1[complex_condition].reset_index(drop=True)

### THIS WILL FILTER SINGLE GENE EVENTS
fraction_results_1 = fraction_results_1[~fraction_results_1["case_control"].apply(count_zeros)]
### THIS WILL FILTER SINGLE GENE EVENTS


fraction_results_1

In [ ]:
# Verify True and False condition indexes
#print(list(complex_condition))

### 5.6 Storing Dataframe
Normalise ratios with the expected ratio (expected mean)

In [ ]:
fraction_results_2 = pd.DataFrame(ratios_vector/expected_ratio, columns=df_case.columns[1:].tolist())
if config["store_data"]:
    fraction_results_2.to_pickle("{0}_{1}_{2}_forgraph.pkl".format(case_count, control_count, iterations))

### 5.7 Plotting the burden events

In [ ]:
#fraction_results_2 = pd.read_pickle("{0}_{1}_{2}_forgraph.pkl".format(case_count, control_count, iterations))
df_99 = pd.DataFrame(["frequency_bin", "99th_quantile"])
fig, axs = plt.subplots(
    len(fraction_results_2.columns),
    1,
    sharex="none",
    tight_layout=False,
    figsize=(12, 24),
    facecolor='xkcd:mint green',
)
i = 0
for frequency_column in fraction_results_2.columns:
    fraction_results_2[frequency_column].dropna(inplace=True)

    if fraction_results_2[frequency_column].any():
        # Average sample normalization enrichment ratios for "likely impactful" and "likely non-impactful" genes
        q_avg = np.divide(
            np.sum(df_case[df_case.gene.isin(rv_genes)][frequency_column]),
            np.sum(df_control[df_control.gene.isin(rv_genes)][frequency_column]),)
        q_avg_control_group = np.divide(
            np.sum(df_case[df_case.gene.isin(neg_control_genes)][frequency_column]),
            np.sum(
                df_control[df_control.gene.isin(neg_control_genes)][frequency_column]),)
        print(
            "Impact group (av-norm. ): {0}, case_genes_enrichment: {1}, control_genes_enrichment: {2}".format(
                frequency_column, q_avg, q_avg_control_group
            )
        )
        fraction_results_2[frequency_column + "log2"] = np.log2(
            fraction_results_2[frequency_column]).dropna()


        # Unused block
        mu, std = sp.norm.fit(fraction_results_2[frequency_column].dropna())
        percentile_99 = np.percentile(fraction_results_2[frequency_column + "log2"].dropna(), 99)
        xmin, xmax = (
            fraction_results_2[frequency_column].min(),
            fraction_results_2[frequency_column].dropna().max(),
        )
        x = np.linspace(mu - 3 * std, mu + 3*std, 100)
        _, bins, _ = axs[i].hist(
            fraction_results_2[frequency_column + "log2"],
            density=False,
            log=False,
            histtype="stepfilled",
            stacked=True,
            bins=500,
        )
        p = sp.norm.pdf(bins, mu, std)
        axs[i].set_title(
            "{0}: mean_enrichment={1} positive_genes={2} negative_genes={3} 99th percentile (purple)={4}".format(
                fraction_results_2[frequency_column].name  + "_log2",
                np.round(fraction_results_2[frequency_column].mean(), 2),
                np.round(np.log2(q_avg), 2),
                np.round(np.log2(q_avg_control_group), 2),
                np.round(percentile_99, 2)
                )
        )
        axs[i].axvline(percentile_99, color="purple")
        df_99[frequency_column] = np.round(percentile_99, 2)
        i +=1
plt.show()
print("Done {0} iterations".format(iterations))


### 5.8 Transform the 99th percentiles into a dataframe

In [ ]:
df_99 = df_99.drop(df_99.index[0])
df_99 = df_99.T
df_99.columns = df_99.iloc[0]
df_99 = df_99[1:]

# See the 99th percentile in log2 for all simulations without the filters applied:
df_99

### 6.0 Plotting high burden events

Take in high value burden events and filter out genes from gene sets that actually contribute to the burden event,
keep only impactful (i.e. above 99th percentile) events and plot the times the gene has appeared in burden events against highest event, keep only genes that appear more than 40 times.

In [ ]:
fraction_results_1

In [ ]:

df = fraction_results_1
df["impactful_bool"] = df.apply(lambda row: row.loc["burden_ratio_norm_log2"] > df_99.loc[row.loc["frequency_bin"]], axis=1) #slow
df = df[df.impactful_bool]

df = df.explode("case_control_gene") # Slow due explode by size of gene_set

df = df[~df["case_control_gene"].apply(lambda x: any(pd.isna(case_control_gene) or case_control_gene == 0 for case_control_gene in x))]
df[["impactful_gene", "impactful_burden"]] = df["case_control_gene"].tolist()
# Add some summary statistics about burden events
df2 = df.merge(df.groupby(["frequency_bin", "impactful_gene"])["burden_ratio"].aggregate([pd.Series.count]).reset_index())
# Filter out rare occurrences of high burden (correlating to miniscule bumps above 99th percentile on the main plot)
df2 = df2[df2["count"]>30]
df2.sort_values("burden_ratio_norm_log2", ascending=False, inplace=True)
df2.drop_duplicates(subset="impactful_gene", inplace=True)
df2.sort_values("frequency_bin", inplace=True)
df

### 6.1 Save high value burden events data to csv

In [ ]:
# Store data
if config["store_data"]:
    df.to_csv("{0}_{1}_{2}.csv".format(case_count, control_count, iterations))
    df.to_pickle("{0}_{1}_{2}.pkl".format(case_count, control_count, iterations))

### 6.2  Plot relevant genes in each bin

In [ ]:
#df = pd.read_pickle("{0}_{1}_{2}.pkl".format(case_count, control_count, iterations))

# Plot data, output is max 12 plots for every frequency bin.
genes = []
for frequency_column in fraction_results_2.columns:
    filtered_df = df2[df2["frequency_bin"] == frequency_column]
    if filtered_df.shape[0] > 0:
        #plt.scatter(data=filtered_df["gene"], x=filtered_df["case_control"], y=filtered_df["burden_ratio"])

        ax = filtered_df.plot(x='count', y='burden_ratio_norm_log2', kind='scatter', figsize=(5, 5),
                              title=frequency_column, legend="impactful_gene", facecolor='white', marker=" ")
        texts = []
        filtered_df[['count', 'burden_ratio_norm_log2', 'impactful_gene']].apply(lambda x: texts.append(ax.text(*x, color="red")), axis=1)
        adjust_text(texts, arrowprops=dict(arrowstyle="-", color='b', lw=0.2), time_lim=5, ax=ax)
        genes.append([frequency_column, texts])
        plt.show()

In [ ]:
# Useful to see g:Profiles for these genes
for name, graph in genes:
    print(name)
    for text in graph:
        print(text.get_text())

filtered_df

In [ ]:
important_genes = ["GLE1", "TWIST1", "FANCB", "RDH12", "NPR3", "FAAH2", "XPNPEP3", "AGA", "DIP2B", "SNIP1", "NCKAP1"]
#df = df[df["impactful_gene"].apply(lambda x: x in important_genes)]

import itertools
for pair in (list(itertools.combinations(df["impactful_gene"], 2))):
    print("{0},{1}".format(pair, sum(set(pair).issubset(set(row)) for row in df['gene'])))

In [ ]:
df2